<style>
h1{
    color:red;
}
</style>

# postgresQL Server with Python

## Library

```
pip install psycopg2
pip install pandas
```

---
## Connect postgresQL Server

In [9]:
import psycopg2
import pandas as pd
import json
db = {
    "database":"lavadurian_db2",
    "host":"127.0.0.1",
    "user":"postgres",
    "password":"apisit250",
    "port":"5432"
}
con = psycopg2.connect(**db)
cur = con.cursor()

## Query Data and make DataFrame

In [10]:
table = 'store_product'
col =  "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{0}' ORDER BY ORDINAL_POSITION;".format(table)
cur.execute(col)
columns_1 = ['product_'+i[0] for i in cur.fetchall()]

col =  "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{0}' ORDER BY ORDINAL_POSITION;".format('store_store')
cur.execute(col)
columns_2 = ['store_'+i[0] for i in cur.fetchall()]

# Table Query
query = "SELECT * FROM {0} INNER JOIN store_store ON store_product.store_id = store_store.id".format(table)
cur.execute(query)
# DataFrame
store_product = pd.DataFrame(cur.fetchall(), columns=columns_1+columns_2)

## Writer Excel files

In [11]:
with pd.ExcelWriter('store_product.xlsx') as writer:
    store_product['product_date_created'] = store_product['product_date_created'].dt.tz_localize(None)
    store_product['product_date_update'] = store_product['product_date_update'].dt.tz_localize(None)
    store_product['store_date_created'] = store_product['store_date_created'].dt.tz_localize(None)
    store_product.to_excel(writer, sheet_name='store_product', index=None)

# Dump Data to JSON files

In [12]:
# json_data = store_product.to_json(orient="records", force_ascii=False)
with open("store_product.json", "w", encoding='utf8') as file:
    json_data = store_product.to_json(orient="records")
    json.dump(json.loads(json_data), file, ensure_ascii=False)
